In [44]:
import os
import glob
from functools import reduce
from multiprocessing import Pool

import numpy as np
import pandas as pd
from tqdm import tqdm
from numba import njit

In [80]:
import os
import glob
from functools import reduce
from multiprocessing import Pool

import numpy as np
import pandas as pd
from tqdm import tqdm
N_CPUS = 4
def get_wap(bid_price, ask_price, bid_size, ask_size):
    return (bid_price * ask_size + ask_price * bid_size) / (
        bid_size + ask_size
    )


def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()


def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return ** 2))


def get_realized_vol(book: pd.DataFrame):
    """Mutable method to get past realized volatility"""

    book["wap1"] = get_wap(
        book.bid_price1, book.ask_price1, book.bid_size1, book.ask_size1
    )

    book["wap2"] = get_wap(
        book.bid_price2, book.ask_price2, book.bid_size2, book.ask_size2
    )

    a1 = (
        book['bid_price1'] * book['ask_size1'] +
        book['ask_price1'] * book['bid_size1']
    )
    a2 = (
        book['bid_price2'] * book['ask_size2'] +
        book['ask_price2'] * book['bid_size2']
    )
    b = (
        book['bid_size1'] + book['ask_size1'] +
        book['bid_size2']+ book['ask_size2']
    )
    book['wap3'] = (a1 + a2)/ b

    book['wap4'] = (book['wap1'] + book['wap2']) / 2

    book.loc[:, "log_return1"] = log_return(book["wap1"])
    book.loc[:, "log_return2"] = log_return(book["wap2"])
    book.loc[:, "log_return3"] = log_return(book["wap3"])
    book.loc[:, "log_return4"] = log_return(book["wap4"])

    # book = book[~book['log_return'].isnull()]
    # to not remove rows both on train and test set
    book = book.fillna(0)

    book = book.merge(
        book[["time_id", "log_return1"]]
        .rename({"log_return1": "vol1"}, axis=1)
        .groupby("time_id")
        .agg(realized_volatility),
        how="inner",
        on="time_id",
    ).merge(
        book[["time_id", "log_return2"]]
        .rename({"log_return2": "vol2"}, axis=1)
        .groupby("time_id")
        .agg(realized_volatility),
        how="inner",
        on="time_id",
    ).merge(
        book[["time_id", "log_return3"]]
        .rename({"log_return3": "vol3"}, axis=1)
        .groupby("time_id")
        .agg(realized_volatility),
        how="inner",
        on="time_id",
    ).merge(
        book[["time_id", "log_return4"]]
        .rename({"log_return4": "vol4"}, axis=1)
        .groupby("time_id")
        .agg(realized_volatility),
        how="inner",
        on="time_id",
    )

    book['wap_diff12'] = book['wap2'] - book['wap1']
    book['wap_diff13'] = book['wap3'] - book['wap1']
    book['wap_diff14'] = book['wap4'] - book['wap1']

    book['vol_rate'] = (book['vol1'] / book['vol2']).fillna(0)
    book['vol_diff'] = (book['vol1'] - book['vol2'])

    return book



# Price features (SLOWEST FUNCTION NEEDS TO IMPROVEEEEEEEEEEEEEEEEE)
def get_stats(df, ind:str, column:str):
    """Get aggregated features from the column provided"""
    stats = pd.concat([df[[ind, column]].groupby(ind).mean().rename({column: f'{column}_mean'}, axis=1),
    df[[ind, column]].groupby(ind).median()
    .rename({column: f'{column}_median'}, axis=1),
    df[[ind, column]].groupby(ind).count()
    .rename({column: f'{column}_count'}, axis=1),
    df[[ind, column]].groupby(ind).min()
    .rename({column: f'{column}_min'}, axis=1),
    df[[ind, column]].groupby(ind).max()
    .rename({column: f'{column}_max'}, axis=1),
    df[[ind, column]].groupby(ind).std()
    .rename({column: f'{column}_std'}, axis=1),
    df[[ind, column]].groupby(ind).quantile(0.25)
    .rename({column: f'{column}_q25'}, axis=1),
    df[[ind, column]].groupby(ind).quantile(0.75)
    .rename({column: f'{column}_q75'}, axis=1),
    df[[ind, column]].groupby(ind).nunique()
    .rename({column: f'{column}_unique'}, axis=1),
    df[[ind, column]].groupby(ind).mad()
    .rename({column: f'{column}_mad'}, axis=1),
    df[[ind, column]].groupby(ind).first()
    .rename({column: f'{column}_first'}, axis=1),
    df[[ind, column]].groupby(ind).last()
    .rename({column: f'{column}_last'}, axis=1)],
    axis=1)

    stats[f'{column}_delta'] = (
        stats[f'{column}_last'] - stats[f'{column}_first']
    )
    stats[f'{column}_delta_abs'] = (
        stats[f'{column}_last'] - stats[f'{column}_first']
    ).abs()
    stats[f'{column}_unique_pct'] = (
        stats[f'{column}_unique'] / stats[f'{column}_count']
    )

    del stats[f'{column}_count']
    del stats[f'{column}_unique']
    del stats[f'{column}_first']
    del stats[f'{column}_last']

    return stats


def get_book_features(files):
    pieces = []
    c = 1
    for f in tqdm(files):
        book = pd.read_parquet(f)

        book = get_realized_vol(book)

        # price
        book['spread1'] = book['ask_price1'] - book['bid_price1']
        book['spread2'] = book['ask_price2'] - book['bid_price2']
        book['ask_spread'] = book['ask_price2'] - book['ask_price1']
        book['bid_spread'] = book['bid_price1'] - book['bid_price2']
        book['cross_spread1'] = book['ask_price1'] - book['bid_price1']
        book['cross_spread2'] = book['ask_price2'] - book['bid_price2']
        book['bas'] = (
            book[['ask_price1', 'ask_price2']].min(axis = 1) /
            book[['bid_price1', 'bid_price2']].max(axis = 1) -
            1
        )

        # size
        book['skew1'] = book['ask_size1'] - book['bid_size1']
        book['skew2'] = book['ask_size2'] - book['bid_size2']
        book['cross_skew1'] = book['ask_size1'] - book['bid_size2']
        book['cross_skew2'] = book['ask_size2'] - book['bid_size1']
        book['ask_sum'] = book['ask_size1'] - book['ask_size2']
        book['bid_sum'] = book['bid_size1'] - book['bid_size2']
        book['skew_whole'] = book['ask_sum'] - book['bid_sum']

        # price - size combinations
        book['bid_volume1'] = book['bid_price1'] * book['bid_size1']
        book['bid_volume2'] = book['bid_price2'] * book['bid_size2']
        book['ask_volume1'] = book['ask_price1'] * book['ask_size1']
        book['ask_volume2'] = book['ask_price2'] * book['ask_size2']

        # sum of volumes
        book_sums = (
            book[['time_id',
                'bid_volume1', 'bid_volume2',
                'ask_volume1','ask_volume2']]
            .groupby('time_id').sum()
            .rename({
                'bid_volume1': 'bid_volume1_sum',
                'bid_volume2': 'bid_volume2_sum',
                'ask_volume1': 'ask_volume1_sum',
                'ask_volume2': 'ask_volume2_sum'
            }, axis=1)
        )
        features_to_get_stats = [
            'wap2', 'wap3', 'wap4',
            'log_return1', 'log_return2', 'log_return3', 'log_return4',
            'vol1', 'vol2', 'vol3', 'vol4', 'vol_diff',
            'wap_diff12', 'wap_diff13', 'wap_diff14',
            'bid_price1', 'bid_price2', 'ask_price1', 'ask_price2',
            'spread1', 'spread2', 'ask_spread', 'bid_spread', 'cross_spread1', 'cross_spread2', 'bas'
        ]
        stats_df = get_stats(book,'time_id','wap1')
        for i in features_to_get_stats:
            stats_df = pd.concat([stats_df,get_stats(book,'time_id',i)])
            print(c)
        # do not merge with not aggregated book
        dfs = [
            book_sums,
            stats_df
        ]
        df_stats = reduce(
            lambda left, right: pd.merge(
                left, right,
                how='inner', left_index=True, right_index=True
            ),
            dfs
        )
        df_stats["stock_id"] = int(f.split("=")[-1])
        df_stats['time_id'] = df_stats.index
        df_stats = df_stats.fillna(method = 'backfill')
        df_stats = df_stats.fillna(method = 'ffill')
#         df_stats.drop('time_id',axis=1,inplace=True)
        df_stats = df_stats.iloc[0,:].reset_index()
        dataset_new = pd.DataFrame(data=[np.array(list(df_stats.iloc[:,1]))],columns = list(df_stats.iloc[:,0]))
        if c == 1:
            data = dataset_new.copy()
            
        else:
            data = pd.concat([data,dataset_new])
        c+=1
        print(c)
        if c >= 4:
            return data , list(data.keys())
    return data , list(data.keys())


def get_trade_features(files):
    """Getting features from trading history"""
    c = 1
    pieces = []

    for f in tqdm(files):
        trades = pd.read_parquet(f)

        trades['trade_volume'] = trades['price'] * trades['size']
        trades['trade_size_per_order'] = (
            trades['size'] / trades['order_count']
        )
        trades['trade_volume_per_order_mean'] = (
            trades['trade_volume'] / trades['trade_size_per_order']
        )

        trades = trades.rename(
            {
                'price': 'trade_price',
                'order_count': 'trade_order_count',
                'seconds_in_bucket': 'trade_seconds_in_bucket'
            },
            axis=1
        )
        
        # sum of volumes, orders and sizes
        trades_sums = (
            trades[['time_id',
                'size', 'trade_order_count', 'trade_volume'
            ]]
            .groupby('time_id').sum()
            .rename({
                'size': 'size_sum',
                'trade_order_count': 'trade_order_count_sum',
                'trade_volume': 'trade_volume_sum'
            }, axis=1)
        )

        # volatility of trades
        trades.loc[:, "trade_log_return"] = log_return(trades["trade_price"])

        trades_vol = (
            trades[["time_id", "trade_log_return"]]
            .rename({"trade_log_return": "trade_vol"}, axis=1)
            .groupby("time_id")
            .agg(realized_volatility)
        )

        features_to_get_stats = ['trade_log_return']
        stats_df = get_stats(trades,'time_id','trade_price')# where operations are located in the bucket
        for i in features_to_get_stats:
            print(c)
            stats_df = pd.concat([stats_df,get_stats(trades,'time_id',i)])
        # do not merge with not aggregated trades
        dfs = [
            trades_sums,
            trades_vol,
            stats_df
        ]

        dfs.append(
            trades[['time_id', 'trade_seconds_in_bucket']]
            .groupby('time_id').count()
            .rename({'trade_seconds_in_bucket': 'n_trades'}, axis=1)
        )

        df_stats = reduce(
            lambda left, right: pd.merge(
                left, right, how='inner', left_index=True, right_index=True
            ),
            dfs
        )


        df_stats["stock_id"] = int(f.split("=")[-1])
#         pieces.append(df_stats)
#     dataset_new = pd.concat(pieces).reset_index()
    
#     features = list(dataset_new.keys())
        df_stats['time_id'] = df_stats.index
        df_stats = df_stats.fillna(method = 'backfill')
        df_stats = df_stats.fillna(method = 'ffill')
#         df_stats.drop('time_id',axis=1,inplace=True)
        df_stats = df_stats.iloc[0,:].reset_index()
        dataset_new = pd.DataFrame(data=[np.array(list(df_stats.iloc[:,1]))],columns = list(df_stats.iloc[:,0]))
        if c == 1:
            data = dataset_new.copy()
        else:
            data = pd.concat([dataset_new,data])
        c+= 1
        if c >= 3:
            return data, list(data.keys())
    return data, list(data.keys())


def mean_encoding(
    dataset: pd.DataFrame,
    means: dict = None,
    stds: dict = None,
    medians: dict = None
):
    """Dataset with stock_id and target columns"""
    if means is None:
        means = (
            dataset[["stock_id", "target"]]
            .groupby("stock_id").mean()
        )
        means = means['target'].to_dict()

        stds = (
            dataset[["stock_id", "target"]]
            .groupby("stock_id").std()
        )
        stds = stds['target'].to_dict()

        medians = (
            dataset[["stock_id", "target"]]
            .groupby("stock_id").median()
        )
        medians = medians['target'].to_dict()

    dataset["stock_id_mean"] = dataset["stock_id"].apply(lambda x: means[x])
    dataset["stock_id_std"] = dataset["stock_id"].apply(lambda x: stds[x])
    dataset["stock_id_median"] = dataset["stock_id"].apply(lambda x: medians[x])
    del dataset['stock_id']

    return dataset, (means, stds, medians)

In [81]:
dataset = pd.read_csv("D:/Kaggle/optiver-realized-volatility-prediction/train.csv")
dataset['row_id'] = dataset['stock_id'].astype(str) + '-' + dataset['time_id'].astype(str)
dataset

,stock_id,time_id,target,row_id
0,0,5,0.004136,0-5
1,0,11,0.001445,0-11
2,0,16,0.002168,0-16
3,0,31,0.002195,0-31
4,0,62,0.001747,0-62
...,...,...,...,...
428927,126,32751,0.003461,126-32751
428928,126,32753,0.003113,126-32753
428929,126,32758,0.004070,126-32758
428930,126,32763,0.003357,126-32763


In [82]:
files = glob.glob(
    "D:/Kaggle/optiver-realized-volatility-prediction/book_train.parquet/*"
)

In [85]:
books, features_book = get_book_features(files)










  0%|                                                                                          | 0/112 [00:00<?, ?it/s]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:220: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:221: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`







  1%|▋                                                                              | 1/112 [02:06<3:54:38, 126.84s/it]

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2









  2%|█▍                                                                             | 2/112 [04:34<4:04:00, 133.10s/it]

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4


In [87]:
dataset_new = pd.merge(
    books,
    dataset[["time_id", "stock_id",'row_id']],
    how="inner",
    on=["time_id", "stock_id"],
)

In [93]:
dataset_new

,bid_volume1_sum,bid_volume2_sum,ask_volume1_sum,ask_volume2_sum,wap1_mean,wap1_median,wap1_min,wap1_max,wap1_std,wap1_q25,...,trade_log_return_min,trade_log_return_max,trade_log_return_std,trade_log_return_q25,trade_log_return_q75,trade_log_return_mad,trade_log_return_delta,trade_log_return_delta_abs,trade_log_return_unique_pct,n_trades
0,23716.371095,24506.294559,22618.740013,27228.657619,1.003725,1.003923,1.001434,1.004920,0.000693,1.003558,...,-0.000721,0.000610,0.000324,-0.000222,0.000295,0.000271,-0.000308,0.000308,1.000000,40.0
1,35867.289062,31272.945312,64039.640625,34861.621094,1.003585,1.003407,1.000780,1.006438,0.001174,1.002761,...,-0.000754,0.000691,0.000322,-0.000244,0.000326,0.000274,-0.000510,0.000510,0.978495,94.0


In [98]:
files_trade = glob.glob(
    "D:/Kaggle/optiver-realized-volatility-prediction/trade_train.parquet/*"
)
trade_stats, features_trade = get_trade_features(files_trade)











  0%|                                                                                          | 0/112 [00:00<?, ?it/s]

1










  1%|▋                                                                                 | 1/112 [00:08<15:22,  8.31s/it]

2


In [99]:
# merging dataset
dataset_new = pd.merge(
    dataset_new,
    trade_stats,
    how='inner', on=['time_id', 'stock_id'],
    left_index=False, right_index=False
)

In [94]:
test_df = dataset_new.copy()
train_df = pd.read_csv('D:/Kaggle/optiver-realized-volatility-prediction/Features.csv')

In [73]:
'trade_price','trade_log_return'

['wap1_std',
 'wap1_mad',
 'wap1_delta_abs',
 'wap2_std',
 'wap2_mad',
 'wap2_delta_abs',
 'wap3_std',
 'wap3_mad',
 'wap3_delta_abs',
 'wap4_std',
 'wap4_mad',
 'wap4_delta_abs',
 'log_return1_std',
 'log_return1_q25',
 'log_return1_q75',
 'log_return1_mad',
 'log_return2_std',
 'log_return2_q25',
 'log_return2_q75',
 'log_return2_mad',
 'log_return3_std',
 'log_return3_q25',
 'log_return3_q75',
 'log_return3_mad',
 'log_return4_std',
 'log_return4_q25',
 'log_return4_q75',
 'log_return4_mad',
 'vol1_mean',
 'vol1_median',
 'vol1_min',
 'vol1_max',
 'vol1_q25',
 'vol1_q75',
 'vol2_mean',
 'vol2_median',
 'vol2_min',
 'vol2_max',
 'vol2_q25',
 'vol2_q75',
 'vol2_mad',
 'vol3_mean',
 'vol3_median',
 'vol3_min',
 'vol3_max',
 'vol3_q25',
 'vol3_q75',
 'vol4_mean',
 'vol4_median',
 'vol4_min',
 'vol4_max',
 'vol4_q25',
 'vol4_q75',
 'vol_diff_mean',
 'vol_diff_median',
 'vol_diff_min',
 'vol_diff_max',
 'vol_diff_q25',
 'vol_diff_q75',
 'vol_diff_mad',
 'wap_diff12_min',
 'wap_diff12_max'

In [95]:
a = list(train_df.drop('target',axis=1).columns)
a.append('row_id')
test_df = test_df[a]

In [96]:
import tensorflow as tf
train_x , train_y = train_df.iloc[:,:-1],train_df.iloc[:,-1]
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(147,)))
model.add(tf.keras.layers.Dense(10,activation='tanh'))
model.add(tf.keras.layers.Dense(5,activation='tanh'))
model.add(tf.keras.layers.Dense(1,activation='tanh'))
model.compile(optimizer='adam',loss=tf.keras.losses.MeanAbsolutePercentageError(),metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [97]:
model.fit(train_x,train_y,batch_size=128,epochs=10)

Epoch 1/10
3351/3351 [==============================] - 4s 935us/step - loss: 145.2436 - root_mean_squared_error: 0.0154
Epoch 2/10
3351/3351 [==============================] - 3s 771us/step - loss: 28.1502 - root_mean_squared_error: 0.0016
Epoch 3/10
3351/3351 [==============================] - 3s 782us/step - loss: 26.6537 - root_mean_squared_error: 0.0016
Epoch 4/10
3351/3351 [==============================] - 3s 773us/step - loss: 25.0503 - root_mean_squared_error: 0.0015
Epoch 5/10
3351/3351 [==============================] - 3s 765us/step - loss: 24.1283 - root_mean_squared_error: 0.0015
Epoch 6/10
3351/3351 [==============================] - 3s 788us/step - loss: 23.2855 - root_mean_squared_error: 0.0015
Epoch 7/10
3351/3351 [==============================] - 3s 997us/step - loss: 22.9237 - root_mean_squared_error: 0.0015
Epoch 8/10
3351/3351 [==============================] - 4s 1ms/step - loss: 22.7336 - root_mean_squared_error: 0.0015
Epoch 9/10
3351/3351 [===================

In [60]:
pred = model.predict(test_df.iloc[:,:-1])
test_df['target'] = pred
test_df[['row_id','target']].to_csv('submission.csv',index=False)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [61]:
test_df

,wap1_std,wap1_mad,wap1_delta_abs,wap2_std,wap2_mad,wap2_delta_abs,wap3_std,wap3_mad,wap3_delta_abs,wap4_std,...,bas_mad,bas_unique_pct,trade_price_std,trade_price_mad,trade_price_delta_abs,trade_log_return_q25,trade_log_return_q75,trade_log_return_mad,row_id,target
0,0.000693,0.000471,0.002298,0.000781,0.00056,0.002331,0.000714,0.000481,0.002309,0.000698,...,0.000154,0.254967,0.000578,0.000457,0.001345,-0.000222,0.000295,0.000271,0-5,0.004273
1,0.001174,0.000951,0.002346,0.001213,0.00099,0.002040,0.001181,0.000958,0.002216,0.001181,...,0.000137,0.271304,0.001034,0.000866,0.001036,-0.000244,0.000326,0.000274,1-5,0.005091
